In [2]:
file_to_analyse = "gtk.csv"

In [3]:
import pandas

df = pandas.read_csv(file_to_analyse, names=["date", "commit", "added", "removed", "filename"])
df

,date,commit,added,removed,filename
0,2021-09-29 05:57:36 +0000,4a684a637112dd8eae14e250f6c48c0f6932368b,66,43,po/id.po
1,2021-09-28 17:36:23 -0700,e77eaa0acedd86371d9d81de3982d8657864915d,2,2,gdk/macos/gdkmacosglcontext.c
2,2021-09-28 17:48:50 -0400,5bc3923bae211780715714e559b24398f6a385b8,1,3,testsuite/tools/simplify-data-3to4/toolbar.exp...
3,2021-09-28 17:48:50 -0400,5bc3923bae211780715714e559b24398f6a385b8,6,7,tools/gtk-builder-tool-simplify.c
4,2021-09-28 17:48:50 -0400,e99ac8f6d831a3fccaffd255bf229a5efe633870,11,13,gtk/gtkbuilder.c
...,...,...,...,...,...
29358,2019-10-02 05:54:12 +0200,abc595b810bf3abf20ec848c6d0a1280455b3f21,2775,3005,po/da.po
29359,2019-10-02 02:06:50 +0200,7d26d3926c1eae70e7234568f28d34e048885d6b,1,1,po/cs.po
29360,2019-10-01 16:13:43 -0300,0e55f7a52fb2ad119384556fc61bac98d2bb3a5f,5,2,gdk/wayland/gdksurface-wayland.c
29361,2019-10-01 17:32:46 +0200,febe128e725051f6a4b9f8386ddea0d2a1ae5139,236,280,po/cs.po


In [4]:
df.groupby(['filename']).size().reset_index(name='counts').sort_values("counts")

,filename,counts
0,.clang-format,1
3446,gtk/theme/Default/assets/slider-vert-scale-has...,1
3447,gtk/theme/Default/assets/slider-vert-scale-has...,1
1354,gsk/resources/glsl/gl_common.vs.glsl,1
3448,gtk/theme/Default/assets/slider-vert-scale-has...,1
...,...,...
1204,gsk/gl/gskglrenderer.c,212
3152,gtk/theme/Adwaita/_common.scss,258
2827,gtk/gtkwindow.c,306
2818,gtk/gtkwidget.c,371


In [18]:
# df.pivot(index="commit", columns="filename", values="added")
pivot = {}
commits = {}

for _, group in df.groupby(['commit']):
    import itertools
    for filename in group["filename"]:
        try:
            commits[filename] += 1
        except KeyError:
            commits[filename] = 1

    for pair in itertools.combinations(group['filename'], 2):
        try:
            pivot[pair] += 1
        except KeyError:
            pivot[pair] = 1

mylist = sorted(((n / (commits[a]), a, b) for (a, b), n in pivot.items()),reverse=1)
mylist


[(1.0, 'tools/gtk4builder.rng', 'tools/updateiconcache.c'),
 (1.0, 'tools/gtk4builder.rng', 'tools/meson.build'),
 (1.0, 'tools/gtk4builder.its', 'tools/updateiconcache.c'),
 (1.0, 'tools/gtk4builder.its', 'tools/meson.build'),
 (1.0, 'tools/gtk4builder.its', 'tools/gtk4builder.rng'),
 (1.0, 'tools/gtk4builder.its', 'tools/gtk4builder.loc'),
 (1.0, 'tools/gtk-query-settings.c', 'tools/updateiconcache.c'),
 (1.0, 'tools/gtk-query-settings.c', 'tools/meson.build'),
 (1.0, 'tools/gtk-query-settings.c', 'tools/gtk4builder.rng'),
 (1.0, 'tools/gtk-query-settings.c', 'tools/gtk4builder.loc'),
 (1.0, 'tools/gtk-query-settings.c', 'tools/gtk4builder.its'),
 (1.0, 'tools/gtk-launch.c', 'tools/updateiconcache.c'),
 (1.0, 'tools/gtk-launch.c', 'tools/meson.build'),
 (1.0, 'tools/gtk-launch.c', 'tools/gtk4builder.rng'),
 (1.0, 'tools/gtk-launch.c', 'tools/gtk4builder.loc'),
 (1.0, 'tools/gtk-launch.c', 'tools/gtk4builder.its'),
 (1.0, 'tools/gtk-launch.c', 'tools/gtk-query-settings.c'),
 (1.0, 'to